In [ ]:
!pip3 install langchain langchain-community langchain-ollama ollama
!pip3 install pandas

In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

import pandas as pd

In [ ]:
#df_gemini_answers = pd.read_csv('qa-base/qa_gemini.csv')

#df_gemini_answers

In [3]:
df_deepseek_answers = pd.read_csv('qa-base/qa_deepseek.csv')

df_deepseek_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",Factual,"Em difamação, a exceção da verdade somente se ..."
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Não,NaN
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\n\nConfigura-se quando alguém:\n-...",Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",Factual,"A pena é detenção, de seis meses a três anos, ..."
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\n\...,Factual,Art. 110 fala sobre o aumento do prazo de pres...
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\n\nDefinição: Privar ...",Input,"A pergunta pede as penalidades relacionadas, m..."
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\n\nS...",Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não,NaN


In [4]:
#Llama 3
model_llama = OllamaLLM(model="llama3")

#OpenAI

## Alucinações de input

In [6]:
def has_input_hallucination_llm(question, answer, model):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer})

    return result

In [7]:
#Alucinação contextual
def has_context_hallucination_llm(question, answer, model):
    contextHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(contextHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer })

    return result

### LLama 3

In [7]:
input_hallucination_veredict = []
model = OllamaLLM(model="llama3")

for index, row in df_deepseek_answers.iterrows():
    question = row['Pergunta']
    answer = row['Resposta']
    input_hallucination_llm = has_input_hallucination_llm(question, answer, model_llama)
    
    input_hallucination_veredict.append({"question":question, "answer":answer, "llm_response":input_hallucination_llm})
    print(input_hallucination_llm)

0.

A resposta gerada está diretamente relacionada ao tema da pergunta e não foge do tópico. A discussão sobre o crime de difamação e a prova da verdade está em perfeita harmonia com a pergunta feita, portanto, não há alucinação de input.
0

A resposta gerada está diretamente relacionada ao tópico da pergunta feita, apresentando informações sobre a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência. Não há divergência do tema ou mudança para um assunto completamente diferente, o que caracteriza alucinação de input.
0

A resposta fornecida está relacionada ao tema da pergunta, especificamente sobre as práticas que configuram o crime de falsificação de documento público. A resposta é precisa e objetiva, não apresentando nenhuma informação irrelevante ou desviando do tópico da pergunta.
0

A resposta gerada é direta e relacionada ao tópico da pergunta feita pelo usuário ("Como o Código Penal trata o crime de da

In [8]:
df_input_veredict = pd.DataFrame(input_hallucination_veredict)

df_input_veredict

,question,answer,llm_response
0,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",0.\n\nA resposta gerada está diretamente relac...
1,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",0\n\nA resposta gerada está diretamente relaci...
2,Quais práticas configuram o crime de falsifica...,"art. 297, CP\n\nConfigura-se quando alguém:\n-...",0\n\nA resposta fornecida está relacionada ao ...
3,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",0\n\nA resposta gerada é direta e relacionada ...
4,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,0.\n\nA resposta gerada atende ao tópico da pe...
5,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",0.\n\nA resposta gerada não foge do tópico da ...
6,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\n\...,0.\n\nA resposta fornecida está diretamente re...
7,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\n\nDefinição: Privar ...",0\n\nA resposta gerada está diretamente relaci...
8,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\n\nS...",0.\n\nA resposta está direcionada ao tópico da...
9,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,0.\n\nA resposta fornecida está diretamente re...


In [9]:
df_input_veredict.to_csv("llm-output/input_hallucination_deepseek.csv")

### GPT

In [ ]:
input_hallucination_veredict = []
#model = 

#for index, row in df_deepseek_answers.iterrows():
#    question = row['Pergunta']
#    answer = row['Resposta']
#    input_hallucination_llm = has_input_hallucination_llm(question, answer, model)    
#    input_hallucination_veredict.append({"question":question, "answer":answer, "llm_response":input_hallucination_llm})
#    print(input_hallucination_llm)